### In this notebook, we will demonstrate how we process and label the data and building models to classify buildings.

In [ ]:
import geopandas as gpd
import gdal
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon, LineString
import os
import matplotlib.pyplot as plt 
import sklearn 
import imblearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import TimeSeriesSplit

from statistics import mean
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn import metrics
from sklearn import datasets

import rasterio
from rasterio import plot as rasterplot


## Data Preprocessing

### Data Loading

In [ ]:
imagery1 = gdal.Open('Imagery/ImageryAOI1.tif')
imagery2 = gdal.Open('Imagery/ImageryAOI2.tif')
imagery3 = gdal.Open('Imagery/ImageryAOI3.tif')
imagery4 = gdal.Open('Imagery/ImageryAOI4.tif')
imagery5 = gdal.Open('Imagery/ImageryAOI5.tif')

### Convert Imagery to Grid
In this step, we use GDAL to split raster data into equal patches. 

In [ ]:
from shapely.geometry import Point, Polygon, LineString
# We examine the dimensions of each imagery.
I1 = imagery1.GetGeoTransform()
I2 = imagery2.GetGeoTransform()
I3 = imagery3.GetGeoTransform()
I4 = imagery4.GetGeoTransform()
I5 = imagery5.GetGeoTransform()
I1,I2,I3,I4,I5 # in m

size1 = imagery1.RasterXSize,imagery1.RasterYSize
size2 = imagery2.RasterXSize,imagery2.RasterYSize
size3 = imagery3.RasterXSize,imagery3.RasterYSize
size4 = imagery4.RasterXSize,imagery4.RasterYSize
size5 = imagery5.RasterXSize,imagery5.RasterYSize

#### Determine the patch size. 
For consistency, we want the same size for all patches in each imagery. However, it is difficult to find a common divider for the dimensions of all 5 imageries. Hence, we select 20 by 20 as this size provides enough information while avoiding taking up too much computing power. In this case, when each imagery is converted into patches, it is cropped slightly on the rightmost side. 

You can adjust the patch below for new sets of imageries. 

In [ ]:
xsize = 20
ysize = 20

#### Slicing and information extraction
Below is a demonstration of how we slice the imagery of AOI1 into patches. The process is the same for other imageries except that you need to change approariate variable names.

In [ ]:
# Coordinates of imagery 1
xmin_1 = I1[0] # top left corner
ymax_1 = I1[3] # top left corner
res_1 = I1[1] # resolution

# Length of imagery 1
xlen_1 = res_1 * imagery1.RasterXSize # RasterXSize: how many pixels in the x direction
ylen_1 = res_1 * imagery1.RasterYSize

# How many patches in each direction
import math
div_x_1 = math.floor(xlen_1/xsize)
div_y_1 = math.floor(ylen_1/ysize)

# Loop to find the coordinates of each patch
xsteps_1 = [xmin_1 + xsize * i for i in range(div_x_1+1)]
ysteps_1 = [ymax_1 - ysize * i for i in range(div_y_1+1)]

# Change the working directory to where you want to store the sliced patches
%cd Patch_Imagery1

# Prepare an output dataframe for storing coordinate and band information of each patch
# Columns should include: which coordinates(4), band info mean (8), band info sd (8), AOI(1), Code(1)
zeros = np.zeros((div_x_1*div_y_1, 22)) 
imagery1_df = pd.DataFrame(zeros, columns=['xmin','ymax','xmax','ymin','B1_m','B2_m','B3_m','B4_m','B5_m','B6_m','B7_m','B8_m','B1_std','B2_std','B3_std','B4_std','B5_std','B6_std','B7_std','B8_std', 'AOI','Code'])

# In this step, we loop through each patch and perform the following actions:
for i in range(div_x_1):
    for j in range(div_y_1):
        
        # 1. Slice patches into tif files and store them in the work directory defined above
        xmin = xsteps_1[i] 
        xmax = xsteps_1[i+1] 
        ymax = ysteps_1[j]  
        ymin = ysteps_1[j+1]
        
        gdal.Translate('imagery1'+'_'+str(i)+'_'+str(j)+'.tif', imagery1,
                      projWin=(xmin,ymax,xmax,ymin))
        
        # 2. Re-read each patch into python
        imagery = gdal.Open('/Users/shurui/Desktop/ALab/Patch_Imagery1/imagery1'+'_'+str(i)+'_'+str(j)+'.tif')

        # 3. Extract band information of each patch
        B1 = pd.DataFrame(imagery.GetRasterBand(1).ReadAsArray().astype(np.float64))
        B2 = pd.DataFrame(imagery.GetRasterBand(2).ReadAsArray().astype(np.float64))
        B3 = pd.DataFrame(imagery.GetRasterBand(3).ReadAsArray().astype(np.float64))
        B4 = pd.DataFrame(imagery.GetRasterBand(4).ReadAsArray().astype(np.float64))
        B5 = pd.DataFrame(imagery.GetRasterBand(5).ReadAsArray().astype(np.float64))
        B6 = pd.DataFrame(imagery.GetRasterBand(6).ReadAsArray().astype(np.float64))
        B7 = pd.DataFrame(imagery.GetRasterBand(7).ReadAsArray().astype(np.float64))
        B8 = pd.DataFrame(imagery.GetRasterBand(8).ReadAsArray().astype(np.float64))

        # 4. Put coordinate information into the dataframe
        imagery1_df.iloc[i*div_y_1+j,0]=xmin
        imagery1_df.iloc[i*div_y_1+j,1]=ymax
        imagery1_df.iloc[i*div_y_1+j,2]=xmax
        imagery1_df.iloc[i*div_y_1+j,3]=ymin
        
        # 5. Take average of band infomation of pixels in each patch and put them into the dataframe
        imagery1_df.iloc[i*div_y_1+j,4] = np.mean(B1).mean()
        imagery1_df.iloc[i*div_y_1+j,5] = np.mean(B2).mean()
        imagery1_df.iloc[i*div_y_1+j,6] = np.mean(B3).mean()
        imagery1_df.iloc[i*div_y_1+j,7] = np.mean(B4).mean()
        imagery1_df.iloc[i*div_y_1+j,8] = np.mean(B5).mean()
        imagery1_df.iloc[i*div_y_1+j,9] = np.mean(B6).mean()
        imagery1_df.iloc[i*div_y_1+j,10] = np.mean(B7).mean()
        imagery1_df.iloc[i*div_y_1+j,11] = np.mean(B8).mean()

        # 6. Take standard deviation of band infomation of pixels in each patch and put them into the dataframe
        imagery1_df.iloc[i*div_y_1+j,12] = B1.values.flatten().std()
        imagery1_df.iloc[i*div_y_1+j,13] = B2.values.flatten().std()
        imagery1_df.iloc[i*div_y_1+j,14] = B3.values.flatten().std()
        imagery1_df.iloc[i*div_y_1+j,15] = B4.values.flatten().std()
        imagery1_df.iloc[i*div_y_1+j,16] = B5.values.flatten().std()
        imagery1_df.iloc[i*div_y_1+j,17] = B6.values.flatten().std()
        imagery1_df.iloc[i*div_y_1+j,18] = B7.values.flatten().std()
        imagery1_df.iloc[i*div_y_1+j,19] = B8.values.flatten().std()

        # 7. Put the associated AOI number into the Dataframe
        imagery1_df.iloc[i*div_y_1+j,20] = 1
        
        # 8. Give each patch a code which might be used later
        imagery1_df.iloc[i*div_y_1+j,21] = '1_'+str(i)+'_'+str(j)



## Labeling
In this step, we will label each patch with either 1 or 0. Below is a demonstration of how we label patches in AOI1. The process is the same for other AOIs except that you need to change approariate variable names.

If the intersection of building and the patch exceeds a threshold, we will label this patch as 1, indicating that there is a building in this patch. 

If the intersection of building and the patch is below a threshold, we will label this patch as 0, indicating that there is no building in this patch. 

The value of the threshold can be defined according to experience or the data distribution.

In [ ]:
# Load the dataframe for this AOI and the building shapefiles
AOI1 = imagery1_df.copy()
buildings12 = gpd.read_file('AOI 1-2/Buildings/Buildings.shp')

# Create a list of polygons for patches in this AOI

# The first polygon of AOI pieces
tl1 = [AOI1.iloc[0,1],AOI1.iloc[0,2]]
tr1 = [AOI1.iloc[0,3],AOI1.iloc[0,2]]
bl1 = [AOI1.iloc[0,1],AOI1.iloc[0,4]]
br1 = [AOI1.iloc[0,3],AOI1.iloc[0,4]]
coords1 = [bl1,br1,tr1,tl1]
polygon1 = Polygon(coords1)
# Define a geoseries to collect polygons (use the first polygon to initiate the geoseries)
polysAOI1 = gpd.GeoSeries([polygon1])

# Loop through the remaining patches to accumulate a geoseries
for j in range(1,len(AOI1)):
    #piece
    tl = [AOI1.iloc[j,1],AOI1.iloc[j,2]]
    tr = [AOI1.iloc[j,3],AOI1.iloc[j,2]]
    bl = [AOI1.iloc[j,1],AOI1.iloc[j,4]]
    br = [AOI1.iloc[j,3],AOI1.iloc[j,4]]
    coords = [bl,br,tr,tl]
    polygon_geom = Polygon(coords)
    polyseries = gpd.GeoSeries([polygon_geom])
    polysAOI1 = polysAOI1.append(polyseries)
    
# Similarly, create a list for building polygons
polysbuilding12 = gpd.GeoSeries([buildings12.iloc[0,2]]) #(use the first polygon to initiate the geoseries)
for i in range(1,len(buildings12)):
    polyb = gpd.GeoSeries([buildings12.iloc[i,2]])
    polysbuilding12 = polysbuilding12.append(polyb)

# Intersection of patch and building
dfAOI1 = gpd.GeoDataFrame({'geometry': polysAOI1, '# of patch':list(range(0,len(AOI1)))})
dfbuildings12 = gpd.GeoDataFrame({'geometry': polysbuilding12, '# of building':list(range(0,len(buildings12)))})
output1 = gpd.overlay(dfAOI1, dfbuildings12, how='intersection')

# Compute the area of intersection
arealist1 = []
for i in range(0,len(output1)):
    area = output1.iloc[i,2].area
    arealist1.append(area)
output1['area'] = arealist1

# If we want to determine the label based on a threshold later, we can output the file here
#output.to_csv('/Users/shurui/Desktop/ALab/output_intersection.csv')

# If we want to determine the label now

# Label determination
label_list = list()
# Define the threshold
threshold = 0.0005
# Loop through each patch to compare the area of intersection with the threshold
for j in range(0,len(AOI1)):
    thispatch = output1.loc[output1['# of patch'] == j]
    # If this patch does not have any intersection, that is, there is no building, the label is 0
    if thispatch.empty == True:
        x = 0
        label_list.append(x)  
    # Sum up the area of intersection of this patch
    totalbuildingarea = thispatch.iloc[:,3].sum() 
    areaofpatch = dfAOI1.iloc[j,0].area
    # Compute the fraction of intersection out of total area of this patch
    ratio = totalbuildingarea/areaofpatch
    # If the fraction exceeds the threshold, label = 1
    if ratio >= threshold:
        x = 1
        label_list.append(x)
     # If the fraction does not exceed the threshold, label = 0
    else:
        x = 0
        label_list.append(x)

# Combine the label information with coordinate and band information
AOI1full = AOI1.copy()
AOI1full['fraction']=label_list1

# Save this output
AOI1full.to_csv('/Users/shurui/Desktop/ALab/AOI1full.csv')

## Modeling
In this step, we will perform 4 models on our labelled data. After examination, we found that there is no building on AOI2. Hence, for class balance, we choose not to use it in our modeling.

In [ ]:
# Load packages and datasets (we've exported datasets after preprocessing)
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import TimeSeriesSplit

from statistics import mean
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn import metrics
from sklearn import datasets

# Load data
df1 = pd.read_csv("/content/drive/MyDrive/ALab_team1/Processed Datasets/AOI1full.csv")
df3 = pd.read_csv("/content/drive/MyDrive/ALab_team1/Processed Datasets/AOI3full.csv")
df4 = pd.read_csv("/content/drive/MyDrive/ALab_team1/Processed Datasets/AOI4full.csv")
df5 = pd.read_csv("/content/drive/MyDrive/ALab_team1/Processed Datasets/AOI5full.csv")
train = pd.concat([df1,df3,df5])
test = df4


#### Determine labels
We choose to determine labels during modeling, but you can determine them in preprocessing as specified above.

In [ ]:
train['Label'] = np.where(train['fraction'] >= 0.02, 1, 0)
test['Label'] = np.where(test['fraction'] >= 0.02, 1, 0)

### Data Rebalance
Because buildings are very concentrated, there is a class imbalance in our dataset: we have too few positive labels and too many negative labels. Hence we will try 2 rebalancing methods.

#### Rebalance

In [ ]:
from imblearn.over_sampling import RandomOverSampler 
ros = RandomOverSampler(random_state=0)
X_train_balanced, y_train_balanced = ros.fit_resample(X_train, y_train)

#### Weighted Decision Trees

In [ ]:
from imblearn.over_sampling import SMOTE 
from collections import Counter

print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

### Logistic Regression

In [ ]:
def evaluate_model(model):
    y_test_pred = model.predict(X_test)
    print('Test set accuracy_score', accuracy_score(y_test, y_test_pred))

    # confusion matrix
    # cm = confusion_matrix(y_test, y_test_pred)

    # tn, fp, fn, tp = cm.ravel()
    # print("tn, fp, fn, tp is", tn, fp, fn, tp)

    fpr, tpr, thresholds = roc_curve(y_test.tolist(), y_test_pred.tolist())
    auc_score = auc(fpr, tpr)
    print("AUC", auc_score)

    print("Test Confusion Matrix:")
    fig = plot_confusion_matrix(model, X_test, y_test, display_labels=['No Buildings', 'Buildings'], cmap='Greens')
    print(fig)

    print(classification_report(y_test, model.predict(X_test)))

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=2000)
logreg.fit(X_train_balanced, y_train_balanced)
y_test_pred = logreg.predict(X_test)
evaluate_model(logreg)
print(classification_report(y_test, logreg.predict(X_test)))

### CART

In [ ]:
## using Over Sampler
model = DecisionTreeClassifier(max_depth=6)
model.fit(X_train_sm,y_train_sm)
evaluate_model(model)

# Decision Tree with cv, sm balanced data
model = DecisionTreeClassifier()
params = {
    'max_depth': [2, 3, 5, 6, 10, 15, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

# cv = GroupShuffleSplit().split(X_train_sm, y_train_sm, 5)
cv = TimeSeriesSplit(n_splits=4, test_size=1, gap=2)
grid_search = GridSearchCV(estimator=model, 
                           param_grid=params, 
                           cv=cv, n_jobs=-1, verbose=1, scoring = "roc_auc")

grid_search.fit(X_train_balanced, y_train_balanced)
model_best = grid_search.best_estimator_
evaluate_model(model_best)

### Random Forest
There are a number of papers that discuss this issue, but I really suggest reading Using Random Forest to learn Imbalanced Data, that proposes the use of Weighted Gini (or Entropy) to take into account the class distribution, or using a mixture of Under and Over sampling of the classes when bagging decision trees.

In [ ]:
SMOTE_SRF = RandomForestClassifier(n_estimators=150, random_state=0)
#Create Stratified K-fold cross validation
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
scoring = ('f1', 'recall', 'precision')

#Evaluate SRF model
scores = cross_validate(SMOTE_SRF, X_train_sm, y_train_sm, scoring=scoring, cv=cv)
#Get average evaluation metrics

print('Mean f1: %.3f' % mean(scores['test_f1']))
print('Mean recall: %.3f' % mean(scores['test_recall']))
print('Mean precision: %.3f' % mean(scores['test_precision']))

SMOTE_SRF.fit(X_train, y_train)

y_pred = SMOTE_SRF.predict(X_test)

fig = plot_confusion_matrix(SMOTE_SRF, X_test, y_test, display_labels=['With Buildings', 'No Buildings'], cmap='Greens')
plt.title('Standard Random Forest Confusion Matrix')
plt.show()
evaluate_model(SMOTE_SRF)

### XGBoost

In [ ]:
import xgboost as xgb
modelxgb = xgb.XGBClassifier(learning_rate=0.01, max_depth=2)
modelxgb.fit(X_train_sm, y_train_sm)
y_pred_xgb = modelxgb.predict(X_test)
confusion_matrix(y_test, modelxgb.predict(X_test))
print(classification_report(y_test, modelxgb.predict(X_test)))

## Visualization

#### Comparison of AUC

In [ ]:
y_pred_1 = model.predict_proba(X_test)[:, 1]
fpr_1, tpr_1, _ = metrics.roc_curve(y_test, y_pred_1)
auc_1 = round(metrics.roc_auc_score(y_test, y_pred_1), 4)

y_pred_2 = logreg.predict_proba(X_test)[:, 1]
fpr_2, tpr_2, _ = metrics.roc_curve(y_test, y_pred_2)
auc_2 = round(metrics.roc_auc_score(y_test, y_pred_2), 4)

y_pred_3 = SMOTE_SRF.predict_proba(X_test)[:, 1]
fpr_3, tpr_3, _ = metrics.roc_curve(y_test, y_pred_3)
auc_3 = round(metrics.roc_auc_score(y_test, y_pred_3), 4)

y_pred_xgb = modelxgb.predict_proba(X_test)[:, 1]
fpr_xgb, tpr_xgb, _ = metrics.roc_curve(y_test, y_pred_xgb)
auc_xgb = round(metrics.roc_auc_score(y_test, y_pred_xgb), 4)

plt.plot(fpr_1, tpr_1, label="Decision Tree, AUC="+str(auc_1))
plt.plot(fpr_2, tpr_2, label="Logistic regression, AUC="+str(auc_2))
plt.plot(fpr_3, tpr_3, label="Random Forest, AUC="+str(auc_3))
plt.plot(fpr_xgb, tpr_xgb, label="XGBoost, AUC="+str(auc_xgb))

plt.legend()

#### Building Labels Visualization

In [ ]:
# Firstly, convert all patches with postive labels into shapefiles
im = gdal.Open('Patch_Imagery4/imagery4_0_0.tif')
X = pd.read_csv("imagery4.csv")
y = pd.read_csv('y_pred_log.csv')
X['Label'] = y
data = X[X['Label']==1]

# the first polygon of AOI pieces
tl1 = [data.iloc[0,1],data.iloc[0,2]]
tr1 = [data.iloc[0,3],data.iloc[0,2]]
bl1 = [data.iloc[0,1],data.iloc[0,4]]
br1 = [data.iloc[0,3],data.iloc[0,4]]
coords1 = [bl1,br1,tr1,tl1]
polygon1 = Polygon(coords1)
# Define a geoseries to collect polygons (use the first polygon to initiate the geoseries)
polys = gpd.GeoSeries([polygon1])

for i in range(1,len(data)):
    
    tl = [data.iloc[i,1],data.iloc[i,2]]
    tr = [data.iloc[i,3],data.iloc[i,2]]
    bl = [data.iloc[i,1],data.iloc[i,4]]
    br = [data.iloc[i,3],data.iloc[i,4]]
    coords = [bl,br,tr,tl]
    polygon_geom = Polygon(coords)
    polyseries = gpd.GeoSeries([polygon_geom])
    polys = polys.append(polyseries)
    
pred = gpd.GeoDataFrame(geometry=polys,crs='epsg:4326')
pred.to_file(filename='pred.shp', driver='ESRI Shapefile')

In [ ]:
# Secondly, visualize
tiff = rasterio.open('Imagery/ImageryAOI4.tif')
tiff_extent = [tiff.bounds[0], tiff.bounds[2], tiff.bounds[1], tiff.bounds[3]]

shapefile = gpd.read_file('pred.shp')
#shapefile = shapefile.to_crs('epsg:4326')

f, ax = plt.subplots()
f.set_size_inches(18.5, 10.5)


# plot DEM
rasterplot.show(
    tiff.read(1),  # use tiff.read(1) with your data
    extent=tiff_extent,
    ax=ax,
)
# plot shapefiles
shapefile.plot(ax=ax, facecolor='white', edgecolor='white',aspect=1)
plt.show()
f.savefig('pred.png', dpi=100)